We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import duckdb
import pandas as pd
import numpy as np

from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a db folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Using DB_PATH /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


Quick sanity check to confirm that the required tables exist before building any outcomes

In [2]:
required_tables = ["team_week_panel", "injuries", "rosters_weekly"]

tables = set(con.execute("SHOW TABLES").df()["name"].tolist())
missing = [t for t in required_tables if t not in tables]

print("Missing tables", missing)

if missing:
    raise RuntimeError("Step 9 cannot run because tables are missing, missing are " + ", ".join(missing))

Missing tables []
